In [1]:
%%writefile src/ArrayTest2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    uint24[5] balance = [255, 65536, 95, 50, 1];
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];

    function setLocalDynamicArr() pure public returns(uint) {
        //uint[] myArr; //error
        //uint[] storage myArr; //error uninitialized storage
        //uint[] memory myArr; //ok. but cannot push
        //uint[] myArr = new uint[](3); //error
        //uint[] storage myArr = new uint[](3); //error conversion from memory to stoarge
        uint[] memory myArr = new uint[](3);
        //myArr.push(11); //error
        myArr[0]=11;
        myArr[1]=12;
        //myArr.push(13); //error
        //myArr[5]=15;
        return myArr.length;
    }
    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);  //dynamic
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    //string is a dynamic array itself, which can not be returned.
    //function setArrMemory() view public returns(string[] memory) {
    function getStringDynamicArrMemory() pure public {
        //array storage not allowed
        //error: string[2] storage places = ["9000", "Sydney"];
        string[2] memory places = ["9000", "Sydney"];
        //array memory push not allowed
        //places.push("Seoul");
        places[0]="Seoul";
        //return places;
    }
    /*returning 'string[] storage' is not allowed
    function getCities1_() view public returns(string[] memory) {
        return cities1;  //can not return stoarge var. 
    }*/
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    function getCities1Length() view public returns(uint) { return cities1.length; }
    function setCities23() public {
        //string my = "Seoul"; //error
        string memory my = "seoul";
        cities2[0]="New York";
        cities2.push(my);
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    function setMathMarks() public {
        //mathMarks.push(100); //push for fixed array not allowed
        //mathMarks=uint8([100,60,95,50,80]); //error: uint8 memory -> uint8
        uint8[5] memory temp = [100, 60, 95, 50, 80];
        //mathMarks = int(temp); //error: uint8[5] memory -> int256
        for (uint i = 0; i < temp.length; i++) {
            //mathMarks[i] = int(temp[i]); //error: uint8 -> int256
            mathMarks[i] = int(int8(temp[i])); //ok: uint8 -> int8 -> int256
        }
    }
    function getMathMarks() view public returns (int[] memory) {
        //return mathMarks;  //can not return storage array: error: int[] storage -> int[] memory
        int[] memory _arr256 = new int[](mathMarks.length);
        for (uint i = 0; i < mathMarks.length; i++) {
            _arr256[i] = int256(mathMarks[i]);
        }
        return _arr256;
    }
    //run setMathMarks() beforehand
    function getMathAbove70_() view public returns(int[] memory) {
        // size is not allocated yet -> invalid opcode error
        int[] memory mathAbove70;
        uint counter = 0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) {
                mathAbove70[counter] = mathMarks[i];
                //mathAbove70.push(mathMarks[i]);
                counter++;
            }
        return mathAbove70;
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        //compute lengthOfMathAbove70
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        //move math marks above 70
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Writing src/ArrayTest2.sol


In [2]:
!solc-windows.exe src/ArrayTest2.sol --combined-json abi > src/ArrayTest2ABI.json

In [3]:
!solc-windows.exe src/ArrayTest2.sol --combined-json bin > src/ArrayTest2BIN.json

In [19]:
%%writefile src/ArrayTest2Deploy.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var _binJson = require('./ArrayTest2BIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=_abiJson.contracts[contractName].abi;
//_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin="0x"+_binJson.contracts[contractName].bin; //ok without "0x"

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1621137}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy()

Overwriting src/ArrayTest2Deploy.js


In [20]:
!node src/ArrayTest2Deploy.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Deploying the contract from 0x996eBe67b327Cd733cB6B986c891328fE112b384
hash: 0xe82ad2a8c73aba4f29c04b2d9eda4ccf00db18dde031b3cf3fde9e005a891a0e
---> The contract deployed to: 0x37952b69D905E28db32E8a568357D2080259E66E
>> gas: 1460190


* gas를 1460189로 바꾸었을 때 오류
```
C:\Users\chohj\Code\201810811\node_modules\web3-core-helpers\src\errors.js:63
        var error = new Error(message);
                    ^

Error: The contract code couldn't be stored, please check your gas limit.
    at Object.TransactionError (C:\Users\chohj\Code\201810811\node_modules\web3-core-helpers\src\errors.js:63:21)
    at Object.ContractCodeNotStoredError (C:\Users\chohj\Code\201810811\node_modules\web3-core-helpers\src\errors.js:72:21)
    at C:\Users\chohj\Code\201810811\node_modules\web3-core-method\src\index.js:387:40
    at processTicksAndRejections (node:internal/process/task_queues:96:5) {
  receipt: {
    transactionHash: '0xc7679600285358e5ada06d01cee511c407724ae060471de6a366cef46d4b2289',
    transactionIndex: 0,
    blockNumber: 3,
    blockHash: '0x50440c1aa08dc07a68916570853412065aaabbd9893a986b94847d87c6797676',
    from: '0x996ebe67b327cd733cb6b986c891328fe112b384',
    to: null,
    cumulativeGasUsed: 1460189,
    gasUsed: 1460189,
    contractAddress: '0xa3165159407f5ac2646F59B9852B3A92aEd83335',
    logsBloom: '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
    status: false,
    effectiveGasPrice: '0x77359400',
    type: '0x0',
    events: {}
  }
}
```

In [22]:
%%writefile src/ArrayTest2Use.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=_abiJson.contracts[contractName].abi;
//_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0x37952b69D905E28db32E8a568357D2080259E66E");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    //await arr.methods.setCities23().send({from: accounts[0]});   //out of gas error
    await arr.methods.setCities23().send({from: accounts[0], gas:1460190});
    arr.methods.getCities2().call().then(console.log);

    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    await arr.methods.setMathMarks().send({from: accounts[0], gas: 1460190});
    //ERROR invalid opcode arr.methods.getMathAbove70().call().then(console.log);
    arr.methods.getMarksArr().call().then(console.log);

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/ArrayTest2Use.js


In [23]:
!node src/ArrayTest2Use.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Account: 0x996eBe67b327Cd733cB6B986c891328fE112b384
Balance before: 999981593606000000000
>> gas: 162354
[ 'New York', '', 'seoul', 'Busan' ]
>> gas: 133310
Balance after: 999981002278000000000
Balance diff: 591327999950848
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]


In [24]:
!node src/ArrayTest2Use.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Account: 0x996eBe67b327Cd733cB6B986c891328fE112b384
Balance before: 999981002278000000000
>> gas: 127611
[ 'New York', '', 'seoul', 'Busan', 'seoul', 'Busan' ]
Balance after: 999980679436000000000
Balance diff: 322841999966208
>> gas: 33810
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]


* 두 번째 실행 때는 동적 배열을 만들어 놓은 상태이기 때문에 gas 비용이 줄어든다.